In [1]:
# IMPORTS
from os import path, mkdir, remove
import re as re
import mmap as mmap
import pandas as pd
import statistics
import numpy as np
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import write_sbml_model
from optlang.symbolics import Zero

In [5]:
# FILES NAMES AND CONSTANTS
BRENDA_INPUT_FILE = './brenda_download.txt'
BRENDA_UNFILTERED_DATA_DIR = './BRENDA_data/unfiltered_dataframes'
KCAT_FILE = BRENDA_UNFILTERED_DATA_DIR + '/k_cat.csv'
SA_FILE = BRENDA_UNFILTERED_DATA_DIR + '/specific_activity.csv'
MW_FILE = BRENDA_UNFILTERED_DATA_DIR + '/molecular_weight.csv'
ORG_NAMES_FILE = BRENDA_UNFILTERED_DATA_DIR + '/org_names.csv'
OPT_PH_FILE = BRENDA_UNFILTERED_DATA_DIR + '/optimum_ph.csv'
PH_RANGE_FILE = BRENDA_UNFILTERED_DATA_DIR + '/ph_range.csv'
OPT_TEMP_FILE = BRENDA_UNFILTERED_DATA_DIR + '/optimum_temp.csv'
TEMP_RANGE_FILE = BRENDA_UNFILTERED_DATA_DIR + '/temp_range.csv'
MNXREF_DB = './universal_metabolic_DB_curated.xlsx' 
BRENDA_FILTERED_DATA_DIR = './BRENDA_data/filtered_dataframes'
FILTERED_KCAT_FILE = BRENDA_FILTERED_DATA_DIR + '/filtered_kcat_df.csv'
FILTERED_SA_FILE = BRENDA_FILTERED_DATA_DIR + '/filtered_sa_df.csv'
FILTERED_MW_FILE = BRENDA_FILTERED_DATA_DIR + '/filtered_mw_df.csv'
FILTERED_ORG_NAMES_FILE = BRENDA_FILTERED_DATA_DIR + '/filtered_org_names_df.csv'
FILTERED_OPT_PH_FILE = BRENDA_FILTERED_DATA_DIR + '/filtered_optimum_ph_df.csv'
FILTERED_PH_RANGE_FILE = BRENDA_FILTERED_DATA_DIR + '/filtered_ph_range_df.csv'
FILTERED_OPT_TEMP_FILE = BRENDA_FILTERED_DATA_DIR + '/filtered_optimum_temp_df.csv'
FILTEREDED_TEMP_RANGE_FILE = BRENDA_FILTERED_DATA_DIR + '/filtered_temp_range_df.csv'
BIGG_TO_METANETX_FILE = './bigg2metanetx.txt'
SEPARATOR = '\t'

# Scrap brenda

## regex for parameters

In [3]:
def split_pr_numbers(pr_numbers):
    # brenda_info = [ec_number, pr_numbers, other info (kcat, sa, mw, etc)]
    splited_pr_numbers = pr_numbers.split(',')
    return splited_pr_numbers

def write_info_on_csv(brenda_info, csv_file):
    r''' 
    Write information extracted from BRENDA_download.txt on a csv file. 
    
    Inputs:
    -------
    brenda_info: array with the information to be written per line 
    csv_file: .csv file's name
    '''
    for info in brenda_info:
        ec_number, pr_numbers, *brenda_info_rest = info
        pr_numbers_list = split_pr_numbers(pr_numbers)
        for pr_number in pr_numbers_list:
            towrite = [ec_number, pr_number, *brenda_info_rest]
            csv_file.write(SEPARATOR.join(towrite) + '\n')
    
def is_mutant(line):
    'mutant' in line

### turnover number (kcat) ###

def regex_for_kcat(ec_file, ec_number):
    r''' 
    Obtain the turnover number (kcat) information using regular expressions,
    it contains the EC number, PR number, kcat value and substrate, it might 
    contain pH and temperature from each reaction, if they're not known appears
    like 'not specified', if the information corresponds to a mutant strain it
    will be skipped.
    
    Inputs:
    -------
    ec_file: BRENDA sub file corresponding only to one EC number information
    ec_number: respective EC number to ec_file
    Output:
    -------
    kcat_values: matrix with kcat information, each row corresponds to a specific
    enzyme in BRENDA database
    '''
    # Pattern looking for: TN  #pr_num#  Kcat_value  {substrate} pH T°
    kcat_pattern = r'^TN.*\#\d*[,\d+]*\# \d+\.\d* \{.*\}.*[\(?pH \d+\.\d*\)?]?.*[\d+°C]?.*'
    regex = re.compile(kcat_pattern, flags=re.IGNORECASE|re.MULTILINE)
    kcat_matches = regex.findall(ec_file)
    kcat_values = []
    for kcat_match in kcat_matches:
        if is_mutant(kcat_match): continue
        # regular pattern: extract #num# Kcat_value and {substrate}
        pr_kcat_substrate_pattern = r'^TN.*\#(\d*[,\d*]*)\# (.*) \{(.*)\}.*'
        regex2 = re.compile(pr_kcat_substrate_pattern, flags=re.IGNORECASE|re.MULTILINE)
        pr_kcat_substrate = regex2.match(kcat_match)               
        # get PR number/kcat value/substrate for every match
        # pr_numbers could be a list of numbers: #1,2,3,4,5#
        pr_numbers = pr_kcat_substrate.group(1) 
        kcat_value = pr_kcat_substrate.group(2)
        substrate = pr_kcat_substrate.group(3)
        # extract pH value if exists
        ph_pattern = r'.*pH (\d+\.\d*).*'
        regex3 = re.compile(ph_pattern, flags=re.IGNORECASE|re.MULTILINE)
        ph = regex3.match(kcat_match)
        # get pH value for every match
        if ph is not None: ph_value = ph.group(1)
        else: ph_value = 'not specified'
        # extract T° value if exists 
        temp_pattern = r'.*(\d\d)°C.*'
        regex4 = re.compile(temp_pattern, flags=re.IGNORECASE|re.MULTILINE)
        temp = regex4.match(kcat_match)
        # get temperature value for every match
        if temp is not None: temp_value = temp.group(1)
        else: temp_value = 'not specified'
        kcat_values.append([ec_number, pr_numbers, kcat_value, substrate, \
                            ph_value, temp_value])
    return kcat_values
        
def get_kcat(ec_file, ec_number):
    with open(KCAT_FILE, 'a+') as kcat_file:
        kcat_info = regex_for_kcat(ec_file, ec_number)
        write_info_on_csv(kcat_info, kcat_file)

### specific activity (sa) ###

def regex_for_sa(ec_file, ec_number): 
    r''' 
    Obtain the specific activity (sa) information using regular expressions,
    it contains the EC number, PR number and sa value, it might 
    contain pH and temperature from each reaction, if they're not known appears
    like 'not specified', if the information corresponds to a mutant strain it
    will be skipped.
    
    Inputs:
    -------
    ec_file: BRENDA sub file corresponding only to one EC number information
    ec_number: respective EC number to ec_file
    Output:
    -------
    sa_values: matrix with sa information, each row corresponds to a specific
    enzyme in BRENDA database
    '''
    # Pattern looking for: SA  #pr_num#  sa_value pH T°
    sa_pattern = r'^SA.*\#\d*[,\d+]*\# [0-9]+\.[0-9]*.*[pH \d+\.\d*]?.*[\d+°C]?.*'
    regex = re.compile(sa_pattern, flags=re.IGNORECASE|re.MULTILINE)
    sa_matches = regex.findall(ec_file)
    sa_values = []
    for sa_match in sa_matches:
        if is_mutant(sa_match): continue
        # extract #num# sa_value
        pr_sa_pattern = r'^SA.*\#(\d*[,\d+]*)\# ([0-9]+\.[0-9]*).*'
        regex2 = re.compile(pr_sa_pattern, flags=re.IGNORECASE|re.MULTILINE)
        pr_sa = regex2.match(sa_match)               
        # get PR number/specific activity value for every match
        # pr_numbers could be a list of numbers: #1,2,3,4,5#
        pr_numbers = pr_sa.group(1) 
        sa_value= pr_sa.group(2)
        # extract pH value if exists
        ph_pattern = r'.*pH (\d+\.\d*).*'
        regex3 = re.compile(ph_pattern, flags=re.IGNORECASE|re.MULTILINE)
        ph = regex3.match(sa_match)
        # get pH value for every match
        if ph is not None: ph_value = ph.group(1)
        else: ph_value = 'not specified'
        # extract T° value if exists 
        temp_pattern = r'.*(\d\d)°C.*'
        regex4 = re.compile(temp_pattern, flags=re.IGNORECASE|re.MULTILINE)
        temp = regex4.match(sa_match)
        # get temperature value for every match
        if temp is not None: temp_value = temp.group(1)
        else: temp_value = 'not specified'
        sa_values.append([ec_number, pr_numbers, sa_value, ph_value, \
                          temp_value])
    return sa_values
        
def get_sa(ec_file, ec_number):
    with open(SA_FILE, 'a+') as sa_file:
        sa_info = regex_for_sa(ec_file, ec_number)
        write_info_on_csv(sa_info, sa_file)

### molecular weight (mw) ###

def regex_for_mw(ec_file, ec_number): 
    r''' 
    Obtain the molecular weight (mw) information using regular expressions,
    it contains the EC number, PR number and protein's molecular weight.
    
    Inputs:
    -------
    ec_file: BRENDA sub file corresponding only to one EC number information
    ec_number: respective EC number to ec_file
    Output:
    -------
    mw_values: matrix with mw information, each row corresponds to a specific
    enzyme in BRENDA database
    '''
    # Pattern looking for: SA  #pr_num#  MW_value
    mw_pattern = r'^MW\s\#\d+[,\d*]*\# \d\d\d*.*'
    regex = re.compile(mw_pattern, flags=re.IGNORECASE|re.MULTILINE)
    # generate a list with all matches
    mw_matches = regex.findall(ec_file)
    mw_values =[]
    for mw_match in mw_matches:
        # extract #num# mw_value
        pr_mw_pattern = r'^MW\s\#(\d*[,\d*]*)\# (\d\d\d*).*'
        regex2 = re.compile(pr_mw_pattern, flags=re.IGNORECASE|re.MULTILINE)
        pr_mw = regex2.match(mw_match)               
        # get PR number/molecular weight value for every match
        # pr_numbers could be a list of numbers: #1,2,3,4,5#
        pr_numbers = pr_mw.group(1)
        mw_value= pr_mw.group(2)
        mw_values.append([ec_number, pr_numbers, mw_value])
    return mw_values
        
def get_mw(ec_file, ec_number):
    with open(MW_FILE, 'a+') as mw_file:
        mw_info = regex_for_mw(ec_file, ec_number)
        write_info_on_csv(mw_info, mw_file)
        
### organisms names (org_names) ###

def regex_for_org_names(ec_file, ec_number): 
    r''' 
    Obtain the organism's names (org_names) information using regular expressions,
    it contains the EC number, PR number and the name of the organism which 
    every protein belongs to.
    
    Inputs:
    -------
    ec_file: BRENDA sub file corresponding only to one EC number information
    ec_number: respective EC number to ec_file
    Output:
    -------
    org_names_list: matrix with org_names information, each row corresponds to a 
    specific enzyme in BRENDA database
    '''
    # Pattern looking for: PR org_name
    org_names_pattern = r'^PR\t\#[0-9]+,*[0-9]*\# \w+ \w+.*'
    regex = re.compile(org_names_pattern, flags=re.IGNORECASE|re.MULTILINE)
    # generate a list with all matches
    org_names_matches = regex.findall(ec_file)
    org_names_list = []
    for org_names_match in org_names_matches:
        # extract #num# org_name
        pr_org_names_pattern = r'^PR\t\#([0-9]+,*[0-9]*)\# (\w+ \w+).*'
        regex2 = re.compile(pr_org_names_pattern, flags=re.IGNORECASE|re.MULTILINE)
        pr_org_names = regex2.match(org_names_match)               
        # get PR numbers/organism's name for every match
        # pr_numbers could be a list of numbers: #1,2,3,4,5#
        pr_numbers = pr_org_names.group(1) 
        org_names = pr_org_names.group(2)
        org_names_list.append([ec_number, pr_numbers, org_names])
    return org_names_list
        
def get_org_names(ec_file, ec_number):
    with open(ORG_NAMES_FILE, 'a+') as org_names_file:
        org_names_info = regex_for_org_names(ec_file, ec_number)
        write_info_on_csv(org_names_info, org_names_file)

### optimum pH (opt_ph) ###

def regex_for_opt_ph(ec_file, ec_number): 
    r''' 
    Obtain the optimum pH value (opt_ph) information using regular expressions,
    it contains the EC number, PR number and the optimum pH value.
    
    Inputs:
    -------
    ec_file: BRENDA sub file corresponding only to one EC number information
    ec_number: respective EC number to ec_file
    Output:
    -------
    opt_phs: matrix with opt_ph values information, each row corresponds to a 
    specific enzyme in BRENDA database
    '''
    # Pattern looking for: SA  #pr_num#  opt_ph_value
    opt_ph_pattern = r'^PHO\s\#\d*[,\d*]*\# [0-9]+\.*[0-9]* [\(]?.*'
    regex = re.compile(opt_ph_pattern, flags=re.IGNORECASE|re.MULTILINE)
    # generate a list with all matches
    opt_ph_matches = regex.findall(ec_file)
    opt_phs = []
    for opt_ph_match in opt_ph_matches:
        # extract #num# opt_ph_value
        pr_opt_ph_pattern = r'^PHO\s\#(\d*[,\d*]*)\# ([0-9]+\.*[0-9]*) [\(]?.*'
        regex2 = re.compile(pr_opt_ph_pattern, flags=re.IGNORECASE|re.MULTILINE)
        pr_opt_ph = regex2.match(opt_ph_match)               
        # get PR number/optimum pH value for every match
        pr_numbers = pr_opt_ph.group(1) # this could be a list of numbers: #1,2,3,4,5#
        opt_ph_value = pr_opt_ph.group(2)
        opt_phs.append([ec_number, pr_numbers, opt_ph_value])
    return opt_phs
        
def get_opt_ph(ec_file,ec_number):
    with open(OPT_PH_FILE, 'a+') as opt_ph_file:
        opt_ph_info = regex_for_opt_ph(ec_file, ec_number)
        write_info_on_csv(opt_ph_info, opt_ph_file)

### pH range (ph_range) ###

def regex_for_ph_range(ec_file, ec_number): 
    r''' 
    Obtain the pH ranges (ph-range) information using regular expressions,
    it contains the EC number, PR number, the lower and the upper bound of pH.
    
    Inputs:
    -------
    ec_file: BRENDA sub file corresponding only to one EC number information
    ec_number: respective EC number to ec_file
    Output:
    -------
    ph_ranges: matrix with pH range values information, each row corresponds to a 
    specific enzyme in BRENDA database
    '''
    # Pattern looking for: SA  #pr_num#  opt_ph_value
    ph_range_pattern = r'^PHR\t\#\d*[,\d*]*\# [0-9]+\.*[0-9]*-[0-9]+,*[0-9]*.*'
    regex = re.compile(ph_range_pattern, flags=re.IGNORECASE|re.MULTILINE)
    #generate a list with all matches
    ph_range_matches = regex.findall(ec_file)
    ph_ranges = []
    for ph_range_match in ph_range_matches:
        # extract #num# ph_range_values
        pr_ph_range_pattern = r'^PHR\t\#(\d*[,\d*]*)\# ([0-9]+\.*[0-9]*)-([0-9]+,*[0-9]*).*'
        regex2 = re.compile(pr_ph_range_pattern, flags=re.IGNORECASE|re.MULTILINE)
        pr_ph_range = regex2.match(ph_range_match)               
        # get PR number/lower bound and upper bound range values for every match
        # pr_numbers could be a list of numbers: #1,2,3,4,5#
        pr_numbers = pr_ph_range.group(1)
        ph_range_lower_bound = pr_ph_range.group(2)
        ph_range_upper_bound = pr_ph_range.group(3)
        ph_ranges.append([ec_number, pr_numbers, ph_range_lower_bound, \
                          ph_range_upper_bound])
    return ph_ranges
        
def get_ph_range(ec_file, ec_number):
    with open(PH_RANGE_FILE, 'a+') as ph_range_file:
        ph_range_info = regex_for_ph_range(ec_file, ec_number)
        write_info_on_csv(ph_range_info, ph_range_file)

### optimum temperature (opt_temp) ###

def regex_for_opt_temp(ec_file, ec_number): 
    r''' 
    Obtain the optimum temperature value (opt_temp) information using regular 
    expressions, it contains the EC number, PR number and the optimum temperature
    value.
    
    Inputs:
    -------
    ec_file: BRENDA sub file corresponding only to one EC number information
    ec_number: respective EC number to ec_file
    Output:
    -------
    opt_temps: matrix with opt_temp values information, each row corresponds to a 
    specific enzyme in BRENDA database
    '''
    # Pattern looking for: SA  #pr_num#  opt_temp_value
    opt_temp_pattern = r'^TO.*\#\d*[,\d*]*\# [0-9][0-9] .*'
    regex = re.compile(opt_temp_pattern, flags=re.IGNORECASE|re.MULTILINE)
    # generate a list with all matches
    opt_temp_matches = regex.findall(ec_file)
    opt_temps = []
    for opt_temp_match in opt_temp_matches:
        # extract #num# opt_temp_value
        pr_opt_temp_pattern = r'^TO.*\#(\d*[,\d*]*)\# ([0-9][0-9]) .*'
        regex2 = re.compile(pr_opt_temp_pattern, flags=re.IGNORECASE|re.MULTILINE)
        pr_opt_temp = regex2.match(opt_temp_match)               
        # get PR number/optimum temperature value for every match
        # pr_numbers could be a list of numbers: #1,2,3,4,5#
        pr_numbers = pr_opt_temp.group(1) 
        opt_temp_value = pr_opt_temp.group(2)
        opt_temps.append([ec_number, pr_numbers, opt_temp_value])
    return opt_temps
        
def get_opt_temp(ec_file, ec_number):
    with open(OPT_TEMP_FILE, 'a+') as opt_temp_file:
        opt_temp_info = regex_for_opt_temp(ec_file, ec_number)
        write_info_on_csv(opt_temp_info, opt_temp_file)

### temperature range (temp_range) ###

def regex_for_temp_range(ec_file, ec_number):
    r''' 
    Obtain the temperature ranges (temp_range) information using regular
    expressions, it contains the EC number, PR number, the lower and the upper 
    bound of temperature.
    
    Inputs:
    -------
    ec_file: BRENDA sub file corresponding only to one EC number information
    ec_number: respective EC number to ec_file
    Output:
    -------
    ph_ranges: matrix with temp_ranges values information, each row corresponds to a 
    specific enzyme in BRENDA database
    '''
    # Pattern looking for: SA  #pr_num#  temp_range_values
    temp_range_pattern = r'^TR\t\#\d*[,\d*]*\# [0-9]+-[0-9]+ .*'
    regex = re.compile(temp_range_pattern, flags=re.IGNORECASE|re.MULTILINE)
    # generate a list with all matches
    temp_range_matches = regex.findall(ec_file)
    temp_ranges = []
    for temp_range_match in temp_range_matches:
        # extract #num# temp_range_value
        pr_temp_range_pattern = r'^TR\t\#(\d*[,\d*]*)\# ([0-9]+)-([0-9]+) .*'
        regex2 = re.compile(pr_temp_range_pattern, flags=re.IGNORECASE|re.MULTILINE)
        pr_temp_range = regex2.match(temp_range_match)               
        # get PR number/lower bound and upper bound range values for every match
        # pr_numbers could be a list of numbers: #1,2,3,4,5#
        pr_numbers = pr_temp_range.group(1) 
        temp_range_lower_bound = pr_temp_range.group(2)
        temp_range_upper_bound = pr_temp_range.group(3)
        temp_ranges.append([ec_number, pr_numbers, temp_range_lower_bound, \
                            temp_range_upper_bound])
    return temp_ranges

        
def get_temp_range(ec_file,ec_number):
    with open(TEMP_RANGE_FILE,'a+') as temp_range_file:
        temp_range_info = regex_for_temp_range(ec_file,ec_number)
        write_info_on_csv(temp_range_info,temp_range_file)


## Files handling

In [4]:
def cleanup_data():
    info_files = [KCAT_FILE , SA_FILE, MW_FILE, ORG_NAMES_FILE, OPT_PH_FILE, \
                  PH_RANGE_FILE, OPT_TEMP_FILE, TEMP_RANGE_FILE]
    for info_file in info_files:
        if path.exists(info_file):
            remove(info_file)
    create_parameters_files()

def create_parameters_files():
    # kcat file
    kcat_file = open(KCAT_FILE, 'w+')
    kcat_col_labels = ['ECnumber', 'Protein_number', 'Kcat', 'Substrate', 'pH'\
                       , 'Temperature']
    kcat_file.write(SEPARATOR.join(kcat_col_labels) + '\n')
    kcat_file.close()
    # specific activity file
    sa_file = open(SA_FILE, 'w+')
    sa_col_labels = ['ECnumber', 'Protein_number', 'Specific_activity', 'pH', \
                     'Temperature']
    sa_file.write(SEPARATOR.join(sa_col_labels) + '\n')
    sa_file.close()
    # molecular weight file
    mw_file = open(MW_FILE, 'w+')
    mw_col_labels = ['ECnumber', 'Protein_number', 'Molecular_weight']
    mw_file.write(SEPARATOR.join(mw_col_labels) + '\n')
    mw_file.close()
    #organisms names file
    org_names_file = open(ORG_NAMES_FILE, 'w+')
    org_col_labels = ['ECnumber', 'Protein_number', 'Organism']
    org_names_file.write(SEPARATOR.join(org_col_labels) + '\n')
    org_names_file.close()
    #optimum ph file
    opt_ph_file = open(OPT_PH_FILE, 'w+')
    opt_ph_col_labels = ['ECnumber', 'Protein_number', 'Optimum_pH']
    opt_ph_file.write(SEPARATOR.join(opt_ph_col_labels) + '\n')
    opt_ph_file.close()
    #ph ranges file 
    ph_range_file = open(PH_RANGE_FILE, 'w+')
    ph_range_col_labels = ['ECnumber', 'Protein_number', 'pH_lower_bound', \
                           'pH_upper_bound']
    ph_range_file.write(SEPARATOR.join(ph_range_col_labels) + '\n')
    ph_range_file.close()
    #optimum temperature file
    opt_temp_file = open(OPT_TEMP_FILE, 'w+')
    opt_temp_col_labels = ['ECnumber', 'Protein_number', 'Optimum_temperature']
    opt_temp_file.write(SEPARATOR.join(opt_temp_col_labels) + '\n')
    opt_temp_file.close()
    #temperature ranges file
    temp_range_file = open(TEMP_RANGE_FILE, 'w+')
    temp_range_col_labels = ['ECnumber', 'Protein_number', 'Temp_lower_bound',\
                             'Temp_upper_bound']
    temp_range_file.write(SEPARATOR.join(temp_range_col_labels) + '\n')
    temp_range_file.close()

    
    
def start_ec_info(line):
    EC_PATTERN = r'ID\t\d+\.\d+\.\d+\.\d+'
    ec_regex = re.compile(EC_PATTERN)
    ec_match = ec_regex.findall(line)
    
    if len(ec_match) > 0: 
        return True
    
def end_ec_info(line):
    TERMINATOR_PATTERN = r'\/\/\/'
    ec_regex = re.compile(TERMINATOR_PATTERN)
    ec_match = ec_regex.findall(line)
    if len(ec_match) > 0: 
        return True
    else:
        return False

def get_ec_number(line):
    EC_PATTERN = r'ID\t(\d+\.\d+\.\d+\.\d+)'
    ec_regex = re.compile(EC_PATTERN)
    ec_match = ec_regex.findall(line)
    return ec_match[0]

def get_ec_info_per_subfile(file):    
    r'''
    Read BRENDA's txt. file and generate a BRENDA subfile by EC number 
    and get all the information using get_ec_info(info, ec_number).
    Every ECnumber's information in BRENDA database is delimited by 
    a first line with the EC number (ID   x.x.x.x) and ending with 
    '///', this function recognized these patterns using regular 
    expressions usig auxiliary functions start_ec_info(line) and 
    end_ec_info(line).
    
    Input:
    ------
    file: BRENDA's .txt file
    '''
    info = ''
    while True:
        line = file.readline()
        if line == '': 
            break
        if start_ec_info(line):
            ec_number = get_ec_number(line)
            info += line
            while not end_ec_info(line):
                line = file.readline()
                info += line
            get_ec_info(info, ec_number)
            info = ''
    
def get_ec_info(ec_file, ec_number):
    r'''
    Obtain all the relevant parameters information from each subfile 
    of BRENDA using auxiliary functions previously described.
    
    Inputs:
    -------
    ec_file: a subfile from BRENDA which corresponds to a unique 
    EC number information
    '''
    get_kcat(ec_file, ec_number)
    get_sa(ec_file, ec_number)
    get_opt_ph(ec_file, ec_number)
    get_opt_temp(ec_file, ec_number)
    get_ph_range(ec_file, ec_number)
    get_temp_range(ec_file, ec_number)
    get_mw(ec_file, ec_number)
    get_org_names(ec_file, ec_number)

def get_info_from_BRENDA():
    r'''
    Obtain all the relevant parameters information from BRENDA using
    auxiliary functions previously described. It also creates a new 
    directory in order to save all de parameter information on a csv
    file format
    '''
    if path.isdir(BRENDA_UNFILTERED_DATA_DIR): 
        cleanup_data()
    else:
        mkdir(BRENDA_UNFILTERED_DATA_DIR)
        create_parameters_files()
    with open(BRENDA_INPUT_FILE, 'r') as brenda_file:
        get_ec_info_per_subfile(brenda_file)

In [6]:
get_info_from_BRENDA()

# Filter dataframes

Catalysis temperature has to be **40°C or less** and **pH of catalysis has to be [4.0-8.0]**

Parameters filtered directly:

 - kcat
 - sa
 - pH range
 - temperature range
 
Parameters filtered undirectly (using EC number an PR number from directly filtered parameters):

 - optimum temperature
 - optimum pH
 - molecular weight
 - organism's names

## Import csv files

In [7]:
#import Kcat
kcat_df = pd.read_csv(KCAT_FILE,index_col=False,sep = '\t',\
                       dtype={'ECnumber': object, 'Protein number': object})

#import SA
sa_df = pd.read_csv (SA_FILE,index_col=False, sep ='\t',\
                     dtype={'ECnumber': object, 'Protein number': object})

#import pH ranges
ph_range_df = pd.read_csv (PH_RANGE_FILE,index_col=False, sep ='\t',\
                           dtype={'ECnumber': object, 'Protein number': object})

#import temperature ranges
temp_range_df = pd.read_csv (TEMP_RANGE_FILE,index_col=False, sep ='\t',\
                             dtype={'ECnumber': object, 'Protein number': object})

#import optimum 
optimum_ph_df = pd.read_csv (OPT_PH_FILE,index_col=False, sep ='\t',\
                             dtype={'ECnumber': object, 'Protein number': object,\
                                    'optimum pH':float})

#import optimum temperature
optimum_temp_df = pd.read_csv (OPT_TEMP_FILE,index_col=False, sep ='\t',\
                               dtype={'ECnumber': object, 'Protein number': object,\
                                      'Optimum temperature':int})

# import organism's names
org_names_df = pd.read_csv (ORG_NAMES_FILE,index_col=False, sep ='\t',\
                            dtype={'ECnumber': object, 'Protein number': object,\
                                   'Organism':object})

#import molecular weight values
mw_df = pd.read_csv (MW_FILE,index_col=False, sep ='\t',\
                     dtype={'ECnumber': object, 'Protein number': object,\
                            'Molecular weight':int})


## Filter kcat values 

In [8]:
## Filtering by pH: ("not specified" must be included)
kcat_with_ph_info = kcat_df.query('pH != "not specified"',inplace = False)
#convert str type to float 
kcat_with_ph_info['pH'] = kcat_with_ph_info['pH'].apply(pd.to_numeric)
#pH must be < 8.0 and > 4.0
ubandlb_kcat_with_ph_info = kcat_with_ph_info.query('(pH <= 8.0) and (pH >= 4.0)'\
                                                    , inplace = False) 
#include "not specified" info
kcat_without_ph_info = kcat_df.query('pH == "not specified"',inplace = False)

#concatenate ph filtered kcats
ph_filtered_kcats = [ubandlb_kcat_with_ph_info,kcat_without_ph_info]
ph_filtered_kcat_df = pd.concat(ph_filtered_kcats,ignore_index=False)

## Filtering by T°: ("not specified" must be included)
kcat_with_temp_info = ph_filtered_kcat_df.query('Temperature != "not specified"',\
                                                inplace = False)
kcat_with_temp_info['Temperature'] = kcat_with_temp_info['Temperature'].apply(pd.to_numeric)
ub_kcat_with_temp_info = kcat_with_temp_info.query(('Temperature <= 40'), \
                                                   inplace = False) 
#include "not specified" info
kcat_without_temp_info = ph_filtered_kcat_df.query('Temperature == "not specified"',\
                                                   inplace = False)

#concatenate temp and ph filtered kcats
temp_filtered_kcat = [ub_kcat_with_temp_info,kcat_without_temp_info]

filtered_kcat_df = pd.concat(temp_filtered_kcat,ignore_index=False)

<ipython-input-8-fcb4488149d7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kcat_with_ph_info['pH'] = kcat_with_ph_info['pH'].apply(pd.to_numeric)
<ipython-input-8-fcb4488149d7>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kcat_with_temp_info['Temperature'] = kcat_with_temp_info['Temperature'].apply(pd.to_numeric)


## Filter Specific activity values

In [9]:
## Filtering by pH: ("not specified" must be included)
sa_with_ph_info = sa_df.query('pH != "not specified"',inplace = False)
#convert str type to float 
sa_with_ph_info['pH'] = sa_with_ph_info['pH'].apply(pd.to_numeric)
#pH must be < 8.0 and > 4.0
ubandlb_sa_with_ph_info = sa_with_ph_info.query('(pH <= 8.0) and (pH >= 4.0)', \
                                                inplace = False) 
sa_without_ph_info = sa_df.query('pH == "not specified"',inplace = False)

#concatenate ph filtered kcats
ph_filtered_sa = [ubandlb_sa_with_ph_info,sa_without_ph_info]
ph_filtered_sa_df = pd.concat(ph_filtered_sa,ignore_index=False)

## Filtering by T°: ("not specified" must be included)
sa_with_temp_info = ph_filtered_sa_df.query('Temperature != "not specified"',\
                                            inplace = False)
sa_with_temp_info['Temperature'] = sa_with_temp_info['Temperature'].apply(pd.to_numeric)
ub_sa_with_temp_info = sa_with_temp_info.query(('Temperature <= 40'), \
                                               inplace = False) 
#include "not specified" info
sa_without_temp_info = ph_filtered_sa_df.query('Temperature == "not specified"',\
                                               inplace = False)

#concatenate df3 and df4
temp_filtered_sa = [ub_sa_with_temp_info,sa_without_temp_info]
filtered_sa_df = pd.concat(temp_filtered_sa,ignore_index=True)

<ipython-input-9-c079957a30aa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa_with_ph_info['pH'] = sa_with_ph_info['pH'].apply(pd.to_numeric)
<ipython-input-9-c079957a30aa>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa_with_temp_info['Temperature'] = sa_with_temp_info['Temperature'].apply(pd.to_numeric)


## Filter pH range lower and upper bound

In [10]:
filtered_ph_range_df = ph_range_df.query(
    '(pH_lower_bound < 1.0 and pH_upper_bound >= 8.0)\
    or (pH_lower_bound <= 4.0 and pH_upper_bound <= 8.0 and pH_upper_bound > 4.0)\
    or (pH_lower_bound >= 4.0 and pH_upper_bound >= 8.0 and pH_lower_bound < 8.0)\
    or (pH_lower_bound >= 4.0 and pH_upper_bound <= 8.0)', inplace = False)

## Filter temperature range lower and upper bound

In [11]:
filtered_temp_range_df = temp_range_df.query('Temp_upper_bound <= 40',\
                                             inplace = False)

## Filter: molecular weight, organism's names, optimum temperatures and optimum pH

In [12]:
# Obtain ECnumber and protein numbers (filtered dataFrames)
filtered_kcat_ec_pr_numbers = filtered_kcat_df.loc[:,['ECnumber','Protein_number']] 
filtered_sa_ec_pr_numbers = filtered_sa_df.loc[:,['ECnumber','Protein_number']] 
filtered_ph_range_ec_pr_numbers = filtered_ph_range_df.loc[:,['ECnumber','Protein_number']] 
filtered_temp_range_ec_pr_numbers = filtered_temp_range_df.loc[:,['ECnumber','Protein_number']] 

# Concatenate in one dataframe
filtered_ec_pr_numbers = [filtered_kcat_ec_pr_numbers,\
                         filtered_sa_ec_pr_numbers,\
                         filtered_ph_range_ec_pr_numbers,\
                         filtered_temp_range_ec_pr_numbers]
filtered_ec_pr_numbers = pd.concat(filtered_ec_pr_numbers,ignore_index=False)

# Drop duplicates
filtered_ec_pr_numbers = filtered_ec_pr_numbers.drop_duplicates(keep='first', inplace=False)

#Filter dataframes based on ECnumber and Protein number (F dataframe) 
filtered_optimum_ph_df = optimum_ph_df.query('(ECnumber in @filtered_ec_pr_numbers.ECnumber) \
                        and (Protein_number in @filtered_ec_pr_numbers.Protein_number)',\
                                             inplace = False)
filtered_optimum_temp_df = optimum_temp_df.query('(ECnumber in @filtered_ec_pr_numbers.ECnumber) \
                        and (Protein_number in @filtered_ec_pr_numbers.Protein_number)',\
                                                 inplace = False)
filtered_org_names_df = org_names_df.query('(ECnumber in @filtered_ec_pr_numbers.ECnumber) \
                        and (Protein_number in @filtered_ec_pr_numbers.Protein_number)',\
                                           inplace = False)
filtered_mw_df = mw_df.query('(ECnumber in @filtered_ec_pr_numbers.ECnumber) \
                        and (Protein_number in @filtered_ec_pr_numbers.Protein_number)',\
                             inplace = False)

## Save filtered dataframes

In [13]:
def cleanup_filtered_data():
    filtered_files = [FILTERED_KCAT_FILE , FILTERED_SA_FILE, FILTERED_MW_FILE, \
                      FILTERED_ORG_NAMES_FILE, FILTERED_OPT_PH_FILE, \
                      FILTERED_PH_RANGE_FILE, FILTERED_OPT_TEMP_FILE, \
                      FILTEREDED_TEMP_RANGE_FILE]
    for filtered_file in filtered_files:
        if path.exists(filtered_file):
            remove(filtered_file)

def save_filtered_dataframes():
    r'''
    Save filtered dataframes in a new directory, which is created if it
    not existing a previous one
    '''
    if path.isdir(BRENDA_FILTERED_DATA_DIR):
        cleanup_filtered_data()
    else:
        mkdir(BRENDA_FILTERED_DATA_DIR)
    filtered_kcat_df.to_csv(FILTERED_KCAT_FILE)
    filtered_sa_df.to_csv(FILTERED_SA_FILE)
    filtered_ph_range_df.to_csv(FILTERED_PH_RANGE_FILE )
    filtered_temp_range_df.to_csv(FILTEREDED_TEMP_RANGE_FILE)
    filtered_optimum_ph_df.to_csv(FILTERED_OPT_PH_FILE )
    filtered_optimum_temp_df.to_csv(FILTERED_OPT_TEMP_FILE)
    filtered_org_names_df.to_csv(FILTERED_ORG_NAMES_FILE)
    filtered_mw_df.to_csv(FILTERED_MW_FILE)
        

In [14]:
save_filtered_dataframes()

# Mapping databases (BRENDA + MetanetX)

Import MetanetX database and join with the information extrected from brenda.

Incorporated parameters:
- kcat
- specific activity
- molecular weight

## Import MetanetX and create new columns

In [3]:
#import Kcat
filtered_kcat_df = pd.read_csv(FILTERED_KCAT_FILE,index_col=0,sep = ',',\
                       dtype={'ECnumber': object, 'Protein number': object})
#import SA
filtered_sa_df = pd.read_csv (FILTERED_SA_FILE,index_col=0, sep =',',\
                     dtype={'ECnumber': object, 'Protein number': object})
#import molecular weight values
filtered_mw_df = pd.read_csv (FILTERED_MW_FILE,index_col=0, sep =',',\
                     dtype={'ECnumber': object, 'Protein number': object,'Molecular weight':int})

In [4]:
#import DB and create Kcat, SA and MW columns  
universal_db = pd.read_excel(MNXREF_DB)
universal_db['Kcat'] = [None] * len(universal_db['EC Number'])
universal_db['Specific_activity'] = [None] * len(universal_db['EC Number'])
universal_db['Molecular_weight'] = [None] * len(universal_db['EC Number'])

## Join BRENDA information with MetanetX database

kcat, sa and mw filtered dataframes are joined with the previously elaborated MetanetX information, based on EC number ID's

In [5]:
def join_databases(brenda_dataframe, db, column):
    for i in db.index:
        ec_number = db.at[i,'EC Number']
        db.at[i,column] = search_in_brenda_data(brenda_dataframe, ec_number, column)

def get_parameters(ec_number, brenda_dataframe, column):
    unique_ec_numbers = brenda_dataframe['ECnumber'].unique()
    if ec_number in unique_ec_numbers:
        parameters = brenda_dataframe[brenda_dataframe['ECnumber'].str.contains(ec_number)]
        values_to_add = list(parameters[column])
    else: 
        # search for first, second and third digits of the EC number
        approx = brenda_dataframe[brenda_dataframe['ECnumber'].str.contains(ec_number[0:4])]
        values_to_add = list(approx[column])
    return values_to_add
        
def search_in_brenda_data(brenda_dataframe, ec_number, column):
    if pd.isna(ec_number): 
        return None
    splited_ec_numbers = ec_number.split(';')
    parameter_values = []
    for actual_ec_number in splited_ec_numbers:
        values_to_add = get_parameters(actual_ec_number, brenda_dataframe, column)
        parameter_values += values_to_add
    if len(parameter_values) > 0: 
        # agregar prom logaritmos
        numpy_param_values = np.array(parameter_values) 
        ln_param_values = np.log(numpy_param_values)
        return np.exp(np.mean(ln_param_values))
        #return statistics.median(parameter_values)
    else: return None


In [6]:
join_databases(filtered_kcat_df, universal_db, 'Kcat')
join_databases(filtered_sa_df, universal_db, 'Specific_activity')
join_databases(filtered_mw_df, universal_db, 'Molecular_weight')

<ipython-input-5-c0020d42e0fc>:28: RuntimeWarning: divide by zero encountered in log
  ln_param_values = np.log(numpy_param_values)


## Reverse reaction specific activity calculation

$$
\Delta G_{rxn} = -R \cdot T \cdot ln(K_{eq}), K = \frac{K^{+}}{K^{-}}
\\
\Rightarrow K^{-} = \frac{K^{+}}{ exp(-\frac{\Delta G_{rxn}}{R \cdot T})}
$$

In [1]:
import pandas as pd
import numpy as np

In [10]:
universal_db = pd.read_excel('./universal_metabolic_DB_curated_with_brenda_info.xlsx', index_col=0)

universal_db['Reverse_specific_activity'] = '' * len(universal_db)
universal_db['Kcat_minus'] = '' * len(universal_db)

R = 8.314/1000 #ideal gases constant [kJ/(mol K)]
T = 298 # temperature(?) [K]
for i in universal_db.index:
    if not pd.isna(universal_db.at[i,'Specific_activity']) and not \
    pd.isna(universal_db.at[i, 'ΔGr (kJ/mol) mean']):
        sa_minus = universal_db.at[i,'Specific_activity'] \
        /np.exp(-1*(universal_db.at[i, 'ΔGr (kJ/mol) mean']/(R*T)))
        universal_db.at[i, 'Reverse_specific_activity'] = sa_minus
    if not pd.isna(universal_db.at[i,'Kcat']) and not \
    pd.isna(universal_db.at[i, 'ΔGr (kJ/mol) mean']):
        kcat_minus = universal_db.at[i,'Kcat'] \
        /np.exp(-1*(universal_db.at[i, 'ΔGr (kJ/mol) mean']/(R*T)))
        universal_db.at[i, 'Kcat_minus'] = kcat_minus

<ipython-input-10-dfd53b32e107>:11: RuntimeWarning: invalid value encountered in double_scalars
  sa_minus = universal_db.at[i,'Specific_activity'] \
<ipython-input-10-dfd53b32e107>:16: RuntimeWarning: divide by zero encountered in double_scalars
  kcat_minus = universal_db.at[i,'Kcat'] \


In [11]:
universal_db

,MetaNetX ID,Equation,Description,Balanced,Name,EC Number,KEGG ID,BiGG ID,ΔGr (kJ/mol) mean,ΔGr (kJ/mol) std,Kcat,Specific_activity,Molecular_weight,Reverse_specific_activity,Kcat_minus
0,MNXR94689,1 MNXM1 + 1 MNXM10 + 1 MNXM101404 = 1 MNXM8 + ...,"1 `H(+)` + 1 `NADH` + 1 `13-oxo-(9Z,11E)-octad...",True,13-HODE:NAD+ oxidoreductase,1.1.1.-,R07058,13HODEOR,NaN,NaN,0.840684,2.275449,58926.559615,,
1,MNXR94691,1 MNXM1 + 1 MNXM10 + 1 MNXM1526 = 1 MNXM2861 +...,1 `H(+)` + 1 `NADH` + 1 `3-hydroxypropanal` = ...,True,"propane-1,3-diol:NAD+ 1-oxidoreductase",1.1.1.202,R03119,13PPDH,22.890395,1.826016,26.093391,9.257964,110542.698298,95275.5,268532
2,MNXR94711,1 MNXM1 + 1 MNXM6 + 1 MNXM911 = 1 MNXM5 + 1 MN...,1 `H(+)` + 1 `NADPH` + 1 `Delta(1)-piperideine...,True,L-pipecolate:NADP+ 2-oxidoreductase,1.5.1.1;1.5.1.21,R02203,1PPDCRc,18.981196,2.239798,6.084752,0.268769,48771.603881,570.953,12926
3,MNXR94712,1 MNXM1 + 1 MNXM10 + 1 MNXM911 = 1 MNXM684 + 1...,1 `H(+)` + 1 `NADH` + 1 `Delta(1)-piperideine-...,True,L-pipecolate:NAD+ 2-oxidoreductase,1.5.1.1;1.5.1.21,R02201,1PPDCRp,17.803187,2.320421,6.084752,0.268769,48771.603881,354.901,8034.71
4,MNXR94722,1 MNXM1 + 1 MNXM1985 = 1 MNXM2 + 1 MNXM2598,"1 `H(+)` + 1 `3'-AMP` = 1 `H2O` + 1 `2',3'-cyc...",True,"2',3'-Cyclic AMP 3'-nucleotidohydrolase",3.1.4.16,R03537,23PDE7pp,-33.529581,4.816595,1.992947,4.110906,73170.876005,5.45162e-06,2.64292e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7752,MNXR114867,1 MNXM22 + 1 MNXM74890 = 1 MNXM4 + 1 MNXM9385,"1 `H2O2` + 1 `phenazine-1,6-dicarboxylate` = 1...",True,NaN,NaN,R11575,NaN,NaN,NaN,NaN,NaN,NaN,,
7753,MNXR114868,1 MNXM162333 = 1 MNXM37861,"1 `(10aS)-10,10a-dihydrophenazine-1-carboxylat...",True,NaN,NaN,R11576,NaN,NaN,NaN,NaN,NaN,NaN,,
7754,MNXR114869,1 MNXM22 + 1 MNXM9013 = 1 MNXM37861 + 1 MNXM4,"1 `H2O2` + 1 `phenazine-1-carboxylate` = 1 `5,...",True,NaN,NaN,R11577,NaN,NaN,NaN,NaN,NaN,NaN,,
7755,MNXR114870,1 MNXM13 + 1 MNXM2 + 1 MNXM32817 + 1 MNXM8 = 2...,1 `CO2` + 1 `H2O` + 1 `1-hydroxyphenazine` + 1...,True,NaN,1.14.13.218,R11578,NaN,0.000000,0.000000,0.441174,0.974013,58984.551357,0.974013,0.441174


In [2]:
universal_db = pd.read_excel('./universal_metabolic_DB_curated_with_brenda_info.xlsx', index_col=0)

In [4]:
universal_db['ΔGr (kJ/mol) mean'].describe()

count    5757.000000
mean      -43.022227
std       144.299685
min     -1332.000000
25%       -24.176992
50%        -0.801973
75%         0.000000
max      4568.300000
Name: ΔGr (kJ/mol) mean, dtype: float64

### Crear una nueva hoja de excel para agregar a la base de optstoic

In [12]:
ec_number_kinetic_db = universal_db[['KEGG ID', 'Kcat', 'Specific_activity', 'Molecular_weight', 'Kcat_minus', 'Reverse_specific_activity']]

In [13]:
ec_number_kinetic_db

,KEGG ID,Kcat,Specific_activity,Molecular_weight,Kcat_minus,Reverse_specific_activity
0,R07058,0.840684,2.275449,58926.559615,,
1,R03119,26.093391,9.257964,110542.698298,268532,95275.5
2,R02203,6.084752,0.268769,48771.603881,12926,570.953
3,R02201,6.084752,0.268769,48771.603881,8034.71,354.901
4,R03537,1.992947,4.110906,73170.876005,2.64292e-06,5.45162e-06
...,...,...,...,...,...,...
7752,R11575,NaN,NaN,NaN,,
7753,R11576,NaN,NaN,NaN,,
7754,R11577,NaN,NaN,NaN,,
7755,R11578,0.441174,0.974013,58984.551357,0.441174,0.974013


In [14]:
ec_number_kinetic_db.to_excel('enzyme_load_info.xlsx',index=False)

In [4]:
import pandas as pd 
import numpy as np

df = pd.read_excel('./enzyme_load_info.xlsx', index_col=False)
df = df.replace(r'^\s*$', np.NaN, regex=True)
df = df.fillna(0)

In [16]:
sa_plus_data = df.query('(sa_plus != 0) and (sa_plus<=1e9) and (sa_plus>=1e-9)')
sa_minus_data = df.query('(sa_minus != 0) and (sa_minus<=1e9) and (sa_minus>=1e-9)')

In [19]:
sa_plus_data.to_excel('sa_plus_data.xlsx',index=False)
sa_minus_data.to_excel('sa_minus_data.xlsx', index=False)

In [15]:
#df.query('sa_minus == 0')

# Enzyme load minimization

In [13]:
def opt_enzimatic_load(model, db, solution=None, frac=1):   
    with model:
        model.solver = 'glpk'
        add_opt_enzimatic_load(model,db,solution,frac)
        new_solution = model.optimize()
        print("Status:", new_solution)
        print("Objective value:", model.objective.value)
        print('\n')
        for var in model.variables:
            print(var.name, ":", var.primal)
        return new_solution


In [14]:
def add_opt_enzimatic_load(model, db, solution=None, frac=1):

    r'''
    Optimize ecoli model including energy load restrictions (E_i)

    This function optimize a given model adding variables and constraints
    that represents the energy load of reactions, which are related with
    turnover number (Kcat) and changing the original objective to sumatory
    of energy loads (sum(E_i)). It looks for minimize this sumatory.
    
    Modified constraints: fluxes - E_i * Kcat- <= v_i <= E_i * Kcat+ 

    Inputs:
    ----------
    model : cobra.Model
        The model to add variables and constraints and objective to.
    db : pandas Dataframe
        The database in dataframe format which has the Kcat values for every
        reaction in the model

    Outputs:
    ----------
    solution: cobra.Solution
        The solution for this new optimization problem, including objective_value
        and fluxes of every reaction
    '''
    # set biomass flux value to v_opt (default problem)
    if solution is None:
        solution = model.slim_optimize()
    v_biomass = 0.8739215069684305
    clean_db = db
    #clean_db = clean_database(model, db)
    prob = model.problem
    reactions = model.reactions
    vars_and_cons = []
    obj_vars = []
    exchanges = model.exchanges
    # change objective to 0
    biomass_constraint = prob.Constraint(model.objective.expression, lb=v_biomass, ub=v_biomass)
    model.objective = prob.Objective(Zero, direction='min', sloppy=True)
    vars_and_cons.append(biomass_constraint)
    for rxn in model.reactions:
        if (rxn in exchanges): continue
        # skip reactions without GPR or GPR = s0001 (including non-enzimatic transport reactions)
        if rxn.gene_reaction_rule is None or rxn.gene_reaction_rule == 's0001': continue   
        v = rxn.flux_expression
        if rxn.id in clean_db.index:
            # change reversibility of irreversible reactions (rxn.rev = False to rxn.rev = True)
            if not rxn.reversibility: rxn.lower_bound = -1000
            # Variable: energy cost of flux_i
            E = prob.Variable('E_' + rxn.id, lb=0, ub=1000)
            vars_and_cons.append(E)
            # v_i <= E_i*K+
            # v_i - E_i*K+ <= 0
            sa_plus = clean_db.at[rxn.id, 'Specific_activity']
            if sa_plus:
                UBi= prob.Constraint(v - E * sa_plus, ub=0, name='cons_plus_' + rxn.id)
                vars_and_cons.append(UBi)
            # - E_i*K- <= v_i
            # 0 <= v_i + E_i*K-
            sa_minus = clean_db.at[rxn.id, 'Reverse_specific_activity']
            if sa_minus:
                LBi = prob.Constraint(v + E * sa_minus, lb=0, name='cons_minus_' + rxn.id)
                vars_and_cons.append(LBi)
            obj_vars.append(E)
    model.add_cons_vars(vars_and_cons)
    # Set linear coefficients in objective (sum(Ei))
    model.objective.set_linear_coefficients({v: 1.0 for v in obj_vars})

    # objective, energy load variables and their constraints
    print(model.objective, end='\n')
    print("------")
    for const in model.constraints:
        print(const, end='\n')
    print("-------")

In [15]:
def clean_database(model,db,correct_ids = None):
    r'''
    Clean database which has the reactions information

    This function recover only present reactions in the model, comparing to
    the initial database, skip exchange and spontaneous reactions too. Then, 
    set database index to BiGG ID's (usefull on add_opt_enzimatic_load()).
    
    Inputs:
    ----------
    model : cobra.Model
        The model to add variables and constraints and objective to.
    db : pandas Dataframe
        The database in dataframe format which has the Kcat values for every
        reaction in the model
    correct_ids: txt file with correct ids from bigg to metanetx databases

    Outputs:
    ----------
    solution: cobra.Solution
        The solution for this new optimization problem, including objective_value
        and fluxes of every reaction
    '''
    if correct_ids is None: correct_ids = pd.read_csv(BIGG_TO_METANETX_FILE, sep='\t',\
                                          names=['Metanetx_ID'], index_col=0)
    ids = []
    exchanges = model.exchanges
    for rxn in model.reactions:
        if (rxn in exchanges): continue
        elif rxn.gene_reaction_rule is None or rxn.gene_reaction_rule == 's0001': continue
        else: ids.append(rxn.id)
    df_ids = pd.DataFrame(ids, columns=['BiGG_ID'])
    # extract from bigg2metanetx only ecoli ids (index column: Bigg IDs, column 1: Metanetx ID's)
    MNXdata = correct_ids.loc[df_ids.BiGG_ID]
    # change MetanetX ID on ecoli-core ids which appears in DB
    auxdf = pd.DataFrame({
        'BiGG_ID':['PFK','PGI','TKT1','FBP','NADH16'],
        'Metanetx_ID':['MNXR106666','MNXR99910','MNXR107105','MNXR96239','MNXR101870']})
    for j in auxdf.index:
        for i in MNXdata.index:
            if i == auxdf.at[j,'BiGG_ID']: MNXdata.at[i, 'Metanetx_ID'] = auxdf.at[j, 'Metanetx_ID'] 
    # drop NaN values
    cleanMNXdata = MNXdata.dropna()
    # set db index using MetaNetX column
    db.set_index('MetaNetX ID', drop=False, inplace=True)
    # clean db: replace nan with None 
    clean_db = db.where((pd.notna(db)), None)
    # get db rows with the same metanetX IDs that appears in ecoli_core model
    model_data = clean_db.loc[cleanMNXdata.Metanetx_ID]
    # add BiGG ID's column
    model_data['BiGG ID'] = cleanMNXdata.index
    # complete MetanetX ID's column 
    model_data['MetaNetX ID'] = model_data.index
    # change index column to BiGG ID's (usefull to search after)
    model_data.set_index('BiGG ID', drop=False, inplace=True)
    # consider rows with a valid Kcat only
    clean_data = model_data[model_data.Kcat.notnull()]
    return clean_data

In [16]:
sbml_modelo = './e_coli_core.xml'
core_model = cobra.io.read_sbml_model(sbml_modelo)
sbml_modelo2 = './iJO1366.xml'
ijo_model = cobra.io.read_sbml_model(sbml_modelo2)

Using license file /home/cata/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-16


In [1]:
opt_enzimatic_load(core_model,universal_db,frac=1)

In [2]:
opt_enzimatic_load(ijo_model,universal_db,frac=1)